<h1 style="color:black" align="center">Представления для текстов</h1>

## Bag of words

* Заводим словарь, состоящий из всех слов в выборке
* Делаем признак-индикатор для каждого слова из словаря
* Можно добавлять n-граммы

https://towardsdatascience.com/from-word-embeddings-to-pretrained-language-models-a-new-age-in-nlp-part-1-7ed0c7f3dfc5

<img src='img/lecture09/1.png'>

### Недостатки:

* Слишком много признаков
* Не учитываем смыслы слов
* Семантически похожие тексты могут иметь очень разные представления

<h1 style="color:#008B8B">1. word2vec</h1>

* Попробуем обучить вектор-представление для каждого слова
* Что потребовать от такого представления?


* Важная идея: если выкинуть слово, то оно должно хорошо восстанавливаться по представлениям соседних слов
* Может применять и при работе с изображениями

<img src='img/lecture09/2.png'>

https://arxiv.org/abs/1301.3781

## Skip-gram model

Вероятность встреить слово $w_O$ рядом со словом $w_I$:

$$\large P(w_O | w_I) = 
\frac{\exp (\langle v_{w_O}^{'}, v_{w_I} \rangle)}
{\sum\limits_{w \in W} \exp(\langle v_{w}^{'}, v_{w_I} \rangle)}$$

* $W$ - словарь

Для каждого слова будем выучивать два вектора:

* $v_w$ - "Центральное" представление слова. В случае, когда слово стоит в центре контекста и по нему предсказыаем соседние слова.
* $v_{w}^{'}$ - "Контекстное" представление слова. Слово стоит вне контекста и его предсказываем по центральному слову.

**Зачем выучивать два вектора?** Мы для каждого слова выучиваем два вектора, если слово является центральным и вектор, если слово стоит в контексте. Так работает лушче, можно было бы выучивать всего один вектор и использовать все без штрихов.

### Разбор формулы:

Cлово $w_O$ находится в контексте слова $w_I$, если между ними расстояние не больше некоторого чилса. Контекстом центарльного слова $w_I$ называется соседние $k$ слов (два слова до, два после).

$P(w_O | w_I)$ - вероятность того, что встретим слово $w_O$ в контексте слова $w_I$. Ну эта вероятность считается по формле выше:

* В числителе берём скалярное произведение центального представления слова $w_I$ (для слова $w_I$ берём вектор, который предсказывает что рядом). Для слова $w_O$ берём контекстное представление. Получаем характеристику соноправленности двух векторов. Берём от этого экспоненту, так как скалярное произведение может быть отрицательным и положительными.

* И делим на сумму экспонент, где берём выходные векторы для всех слов. Это для нормировки, чтобы сумма всех вероятностей была равна 1, это просто SoftMax. Чем более соноправлены векторы двух слов, тем выще верояность того, что они встретятся вместе.

При максимизации экспонента от скалярного произведения должна быть как можно больше, значит мы требуем чтобы само скалярное произведение было как можно больше. И при максимизации скалаярного произведения мы требуем соноправленности векторов. Если два слова встречаются рядом, то их векторы будут как можно более соноправлены.

### Функционал

Функционал для текста $T = (w_1, w_2, \ldots, w_n)$ из $n$-слов:

$$\large \sum\limits_{i=1}^{n} \sum\limits_{-c \le j \le c \\ \quad j\ne 0 \\ 1 \le i + c \le n} \log P(w_{i + j} | w_j) \to \max$$

* Суммируем по всем словам в тексте, берём $i$-е слово.
* Суммируем по контексту, где $c$ - это размер контекста.
* Под суммой стоит вероятность встретить слово $w_{i +j}$ в контексте слова $w_i$, берём логарифм этой вероятности. 

Простоми словами, берём $i$ слово и требуем, чтобы вероятность встреить другие слова в его контексте была максимальна, только для слов из контекста.

Максимизуерм по векторам слов $\large v_{w_1}^{'}, \ldots, v_{w_n}^{'}$ и по $\large v_{w_1}, \ldots, v_{w_n}$. Подбираем векторы слов так, чтобы вероятность встретить слово в его контексте была как можно больше.

**Почему логарифм?** Мы записываем правдоподобие выборки, для этого небоходимо перемножать вероятности. А для того, чтобы мы их суммировали необходимо вязть логарифм.